Antrenare K-NN vs Nav-Bayes

In [25]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd

Stabilirea caii de acces la Setul de Date Path:

In [26]:
#Folder paths
dataset_folder='/content/drive/MyDrive/Colab Notebooks/ds'

In [27]:
# Function to load images and labels from the dataset
def load_dataset(root_folder):
  images = []
  labels = []
  class_mapping = {} # To map class names to numeric labels
  #allowed_extensions = {'.jpg', '.jpeg', '.png'} # Allowed image extensions


  for class_label, class_name in enumerate(os.listdir(root_folder)):
      class_mapping[class_label] = class_name
      class_folder = os.path.join(root_folder, class_name)

      for filename in os.listdir(class_folder):
          image_path = os.path.join(class_folder, filename)
          image = imread(image_path)

          #ensure image is rgb or grayscale and resize
          if len(image.shape) == 3: #rgb
            image=resize(image,(64,64,3),anti_aliasing=True)
          else: #grayscale
            image=resize(image,(64,64),anti_aliasing=True)

          images.append(image.flatten()) # Flatten the image
          labels.append(class_label)

  return np.array(images), np.array(labels), class_mapping

FUCTIA pentru Salvarea Rezultatelor in fisier CSV

In [28]:
#Function to save results to a CSV file

def save_results(algorithm, y_true, y_pred, class_mapping):
  results_df = pd.DataFrame({'True Label': [class_mapping[label]
                                            for label in y_true],
                            'Predicted Label': [class_mapping[label]
                                                for label in y_pred]})

  results_folder = '/content/drive/MyDrive/Colab Notebooks/results'
  if not os.path.exists(results_folder):
        os.makedirs(results_folder)

  results_file = os.path.join(results_folder, f'{algorithm}_results.csv')
  results_df.to_csv(results_file, index=False)

  print(f'Results saved for {algorithm} at {results_file}')

Functia pentru Salvarea Predictiilor pentru Imaginile de TEST

In [33]:
# Function to save test images with predictions
def save_test_images(algorithm, X_test, y_true, y_pred, class_mapping,
                     probabilities):
  results_folder = '/content/drive/MyDrive/Colab Notebooks/results'
  if not os.path.exists(results_folder):
    os.makedirs(results_folder)

  for idx, (true_label, pred_label,
            prob_dist) in enumerate(zip(y_true, y_pred, probabilities)):
    image = X_test[idx].reshape(64, 64, -1) # Reshape flattened image
    plt.imshow(image)
    plt.title(f'True Label: {class_mapping[true_label]}\nPredicted Label: {class_mapping[pred_label]}')
    plt.savefig(os.path.join(results_folder,
                             f'{algorithm}_test_image_{idx}.png'))
    plt.close()

    # Plot probability distribution
    plt.bar(knn_classifier.classes_, prob_dist, color='blue')
    plt.title(f'Probability Distribution - Test Image {idx}')
    plt.xlabel('Class')
    plt.ylabel('Probability')
    plt.savefig(os.path.join(results_folder,
                             f'{algorithm}_probability_distribution_{idx}.png'))
    plt.close()

  print(f'Test images saved for {algorithm} in {results_folder}')

Procesarea setului de date

In [30]:
# Split the dataset into training and testing sets
X, y, class_mapping = load_dataset(dataset_folder)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, test_size=0.2,
                                                    random_state=42)

Rezultatelor Predictie cu k-NN pentru Imaginile de TEST

In [31]:
# Example 1: k-Nearest Neighbors (k-NN)
# Initialize k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_test_pred_knn = knn_classifier.predict(X_test)



In [34]:
# Save results and test images for k-NN
save_results('knn', y_test, y_test_pred_knn, class_mapping)
probabilities_knn = knn_classifier.predict_proba(X_test)
save_test_images('knn', X_test, y_test,
                 y_test_pred_knn, class_mapping, probabilities_knn)

Results saved for knn at /content/drive/MyDrive/Colab Notebooks/results/knn_results.csv
Test images saved for knn in /content/drive/MyDrive/Colab Notebooks/results


In [35]:
# Example 2: Naive Bayes (Gaussian Naive Bayes)
# Initialize Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the model
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_test_pred_nb = nb_classifier.predict(X_test)

In [36]:
# Save results and test images for Naive Bayes
save_results('naive_bayes', y_test, y_test_pred_nb, class_mapping)
probabilities_nb = nb_classifier.predict_proba(X_test)
save_test_images('naive_bayes', X_test,
                 y_test, y_test_pred_nb,
                 class_mapping, probabilities_nb)

Results saved for naive_bayes at /content/drive/MyDrive/Colab Notebooks/results/naive_bayes_results.csv
Test images saved for naive_bayes in /content/drive/MyDrive/Colab Notebooks/results
